# Assignment 6 

**Course:** AM5801 Computational Lab  
**Author:** Atharv Shete  

## Problem 1 – Smart Metro Navigator (BFS)

### 1.1 Solution notes
- Build a sorted adjacency list to keep traversal deterministic for reporting.
- Perform BFS once, accumulating both distance and path-count information per station.
- Store predecessors during BFS so a single backtrack yields one concrete route from source to destination.
- Surface human-friendly output (including $\infty$ symbols) directly from the BFS artefacts.


### 1.2 Implementation plan & function contracts
- `metro_navigation(n, edges, source, target)`: builds the adjacency list, runs BFS, reconstructs a representative shortest path, and counts all shortest routes.
- Returns:
  - `distance`: raw stop counts where `-1` marks unreachable stations.
  - `distance_display`: formatted view using $\infty$ for unreachable stations.
  - `path`: list of station indices forming one shortest route from `source` to `target`.
  - `path_count`: integer count of distinct shortest routes between the endpoints.

The demo cell feeds the sample network through this function and prints the results for inspection.

In [13]:
from collections import deque
from typing import Dict, Iterable, List, Sequence, Tuple


def metro_navigation(
    n: int,
    edges: Iterable[Tuple[int, int]],
    source: int,
    target: int,
) -> Dict[str, Sequence[int] | List[int] | int | str | List[str]]:
    """Compute BFS-derived navigation data for the metro network in a single routine."""
    if n <= 0:
        raise ValueError("Number of stations must be positive.")
    if not (0 <= source < n and 0 <= target < n):
        raise ValueError("Source and target must be valid station indices.")

    adjacency: List[List[int]] = [[] for _ in range(n)]
    for u, v in edges:
        if not (0 <= u < n and 0 <= v < n):
            raise ValueError(f"Edge ({u}, {v}) references a station outside [0, {n - 1}].")
        adjacency[u].append(v)
        adjacency[v].append(u)
    for neighbours in adjacency:
        neighbours.sort()

    distance = [-1] * n
    parent = [-1] * n
    path_counts = [0] * n

    queue: deque[int] = deque()
    distance[source] = 0
    path_counts[source] = 1
    queue.append(source)

    while queue:
        current = queue.popleft()
        for neighbour in adjacency[current]:
            if distance[neighbour] == -1:
                distance[neighbour] = distance[current] + 1
                parent[neighbour] = current
                path_counts[neighbour] = path_counts[current]
                queue.append(neighbour)
            elif distance[neighbour] == distance[current] + 1:
                path_counts[neighbour] += path_counts[current]

    path: List[int] = []
    if distance[target] != -1:
        cursor = target
        while cursor != -1:
            path.append(cursor)
            if cursor == source:
                break
            cursor = parent[cursor]
        path.reverse()
    distance_display = [dist if dist >= 0 else "∞" for dist in distance]
    return {
        "distance": distance,
        "distance_display": distance_display,
        "path": path,
        "path_count": path_counts[target] if distance[target] != -1 else 0,
    }


### 1.3 Sample demonstration & verification
The following run exercises the sample metro network from the prompt, confirming that the
distances, one shortest route, and the total count of shortest routes match expectations.

In [14]:
sample_n = 6
sample_edges = [
    (0, 1), (0, 2), (1, 3), (2, 3), (3, 4), (4, 5), (2, 5)
]
sample_source = 0
sample_target = 5

summary = metro_navigation(sample_n, sample_edges, sample_source, sample_target)

print("Minimum stops from station 0 to every station:")
for station, dist in enumerate(summary["distance_display"]):
    print(f"  Station {station}: {dist}")

if summary["path"]:
    path_str = " → ".join(map(str, summary["path"]))
    print(f"\nShortest route from {sample_source} to {sample_target}: {path_str}")
    print(f"Number of distinct shortest routes: {summary['path_count']}")
else:
    print(f"\nDestination {sample_target} is unreachable from {sample_source}.")


Minimum stops from station 0 to every station:
  Station 0: 0
  Station 1: 1
  Station 2: 1
  Station 3: 2
  Station 4: 3
  Station 5: 2

Shortest route from 0 to 5: 0 → 2 → 5
Number of distinct shortest routes: 1


## Problem 2 – Maze Treasure Hunt (DFS)

### 2.1 Solution notes
- Validate the grid and starting/ending cells before exploring to short-circuit impossible cases quickly.
- Use an explicit stack for DFS so we can capture full path histories without relying on Python recursion depth.
- Track visited cells via immutable `frozenset`s attached to each stack frame, ensuring path-specific footprints for accurate counting.
- Record the first successful path encountered while still counting all possible simple routes.

### 2.2 Implementation plan & function contracts
- `maze_paths(maze, start, goal)`: iteratively explores the maze and returns
  - `exists`: indicates if at least one valid route reaches the treasure.
  - `path`: coordinates representing the first path discovered.
  - `path_count`: total number of simple paths found.

The demonstration cell applies the function to the sample grid and displays the computed metrics.

In [15]:
from typing import Dict, List, Sequence, Tuple

Coordinate = Tuple[int, int]


def maze_paths(
    maze: Sequence[Sequence[int]],
    start: Coordinate,
    goal: Coordinate,
) -> Dict[str, object]:
    """Enumerate all simple paths in the maze using a single iterative DFS routine."""
    if not maze:
        raise ValueError("Maze must contain at least one row.")

    row_lengths = {len(row) for row in maze}
    if len(row_lengths) != 1:
        raise ValueError("Maze rows must all have the same length.")

    rows = len(maze)
    cols = row_lengths.pop()

    sr, sc = start
    gr, gc = goal
    if not (0 <= sr < rows and 0 <= sc < cols and maze[sr][sc] == 0):
        return {"exists": False, "path": [], "path_count": 0}
    if not (0 <= gr < rows and 0 <= gc < cols and maze[gr][gc] == 0):
        return {"exists": False, "path": [], "path_count": 0}

    stack: List[Tuple[Coordinate, List[Coordinate], frozenset[Coordinate]]] = [
        (start, [start], frozenset({start}))
    ]
    first_path: List[Coordinate] = []
    path_count = 0
    directions = ((1, 0), (-1, 0), (0, 1), (0, -1))

    while stack:
        cell, path, visited = stack.pop()
        if cell == goal:
            path_count += 1
            if not first_path:
                first_path = path
            continue
        r, c = cell
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            neighbour = (nr, nc)
            if (
                0 <= nr < rows
                and 0 <= nc < cols
                and maze[nr][nc] == 0
                and neighbour not in visited
            ):
                stack.append((neighbour, path + [neighbour], visited | {neighbour}))

    return {
        "exists": path_count > 0,
        "path": first_path,
        "path_count": path_count,
    }


### 2.3 Sample demonstration & verification
We now test the DFS routine on the 4×4 maze from the prompt to confirm the path existence,
one valid route, and the total count of simple paths.

In [16]:
maze_example = [
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 0],
    [0, 1, 0, 0],
]
start_cell = (0, 0)
goal_cell = (3, 3)

result = maze_paths(maze_example, start_cell, goal_cell)
path_str = " → ".join(f"({r}, {c})" for r, c in result["path"]) if result["path"] else "(no path)"

print("Primary maze scenario:")
print(f"  Path exists? {result['exists']}")
print(f"  One valid path: {path_str}")
print(f"  Total number of distinct paths: {result['path_count']}")


Primary maze scenario:
  Path exists? True
  One valid path: (0, 0) → (1, 0) → (2, 0) → (2, 1) → (2, 2) → (2, 3) → (3, 3)
  Total number of distinct paths: 3
